In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../')) # open parent directory

## QICK LIBRARY
from qick import *

## tProc Assembler
import qick.tprocv2_assembler as qasm
from importlib import reload  # Python 3.4+
import numpy as np
import unittest
##### Load FGPA BitStream

##################
soc = QickSoc('../qick_216_qp.bit')

PMEM, DMEM, WMEM = 1,2,3

def load_assembler_str(assembler, tproc):
    # PARSE ASM #
    reload(qasm)
    qasm.Logger.level = qasm.Logger.ERROR+1
    p_txt, p_bin = qasm.Assembler.str_asm2bin(assembler)
    p_bin = np.array(p_bin, dtype='int32')
    if len(p_bin) > 0:
        tproc.Load_PMEM(p_bin)
    return len(p_bin) > 0

def clear_tproc_memories(tproc):
    tproc.reset()
    write_buffer = np.zeros(shape=(PROGRAM_MEMORY_SIZE, 8), dtype='int32')
    tproc.Load_PMEM(write_buffer)
    
    write_buffer = np.zeros(shape=(DATA_MEMORY_SIZE, 8), dtype='int32')
    tproc.load_mem(DMEM, buff_in=write_buffer, addr=0)
    
    write_buffer = np.zeros(shape=(WAVE_MEMORY_SIZE, 8), dtype='int32')
    tproc.load_mem(WMEM, buff_in=write_buffer, addr=0)
    
def print_fail(failures, errors):
    DECORATOR_START    = '='*30
    DECORATOR_END      = '='*30 + '\n'*3
    DECORATOR_FAILURES = '----------------FAILURES----------------'
    DECORATOR_ERRORS   = '----------------ERRORS----------------'
    SEPARATOR_FAILURES = '*'*30
    SEPARATOR_ERRORS   = '*'*30
    
    print(DECORATOR_START)
    print(f"FAILURES <{len(failures)}>\tERRORS <{len(errors)}>")

    print(DECORATOR_FAILURES)
    for idx, failure in enumerate(failures):
        if idx > 0: print(SEPARATOR_FAILURES)
        print('\033[94m', failure[0], '\033[0m')
        print(failure[1].strip())

    print(DECORATOR_ERRORS)
    for idx, err in enumerate(errors):
        if idx > 0: print(SEPARATOR_ERRORS)
        print('\033[94m', err[0], '\033[0m')
        print(err[1].strip())

    print(DECORATOR_END)

def test_class(class_name):
    runner = unittest.TextTestRunner(stream=open(os.devnull, 'w'))
    alltests = unittest.TestLoader().loadTestsFromTestCase(class_name)
    
    result = runner.run(alltests)
    if len(result.failures) or len(result.errors):
        print_fail(result.failures, result.errors)
    print(f"Finished testing {class_name}")
        
        
def test_method(method):
    runner = unittest.TextTestRunner(stream=open(os.devnull, 'w'))
    suite = unittest.TestSuite()
    suite.addTest(method)
    
    result = runner.run(suite)
    if len(result.failures) or len(result.errors):
        print_fail(result.failures, result.errors)
    print(f"Finished testing {method}")

def remove_comments(txt):
    tout = txt
    comment_idx = tout.find("//")
    if comment_idx != -1: tout = tout[:comment_idx]
    return tout

def separate_instructions(txt):
    return "\n".join([instruction.strip() for instruction in txt.split(";")])
    
class BadAssemblerException(Exception): pass

In [ ]:
filename = "bad.asm" 
lines = []
with open(filename) as f:
    lines = f.readlines()

lines = map(remove_comments, lines)
lines = map(separate_instructions, lines)
lines = filter(len, lines)
TEST_CASES = list(lines)

print(f"Starting reading \"{filename}\" test cases")
print()
for asm in TEST_CASES:
    try:
        if bool(load_assembler_str(asm, soc.tproc)):
            raise BadAssemblerException(f"-----Assembler did not fail-----\n{asm}\n")
    except BadAssemblerException as e:
        print(e)
print(f"Finished reading \"{filename}\" test cases")